In [113]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy 
import pickle
from glob import glob
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from statistics import mean,stdev


"""Change to the data folder"""
new_path = "../new_train/"
val_path = "../new_val_in/"

# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

### Create a dataset class 

In [114]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
#         reduce_mem_usage(data)
        if self.transform:
        
            data = self.transform(data)

        return data

# def reduce_mem_usage(df, verbose=True):
#     numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
#     columns = ['p_in','v_in','v_out','lane','lane_norm']
#     for col in columns:
#             c_min = df[col].min()
#             c_max = df[col].max()
#             print("C_min",c_min," ",np.iinfo(np.int8).min)
#             print("c_max",c_max," ",np.iinfo(np.int8).max)
#             if True:
#                 if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
#                     print("true")
#                     df[col] = df[col].astype(np.int8)
#                 elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
#                     df[col] = df[col].astype(np.int16)
#                 elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
#                     df[col] = df[col].astype(np.int32)
#                 elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
#                     df[col] = df[col].astype(np.int64)  
#             else:
#                 if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
#                     df[col] = df[col].astype(np.float16)
#                 elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
#                     df[col] = df[col].astype(np.float32)
#                 else:
#                     df[col] = df[col].astype(np.float64)    
#     return df
# intialize a dataset

# transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.5, 0.5, 0.5],
#                          std=[0.5, 0.5, 0.5])
# ])
train_dataset  = ArgoverseDataset(data_path=new_path)
val_dataset = ArgoverseDataset(data_path=val_path)
#print((val_dataset[0]))
#print(len(train_dataset[0]))

### Create a loader to enable batch processing

In [ ]:
batch_sz = 4

def my_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
#     if(len(str(maxVal)) > 3):
#         print(len(maxVal))
#     print("maxStringLength",len(str(maxVal)))
#     print("k   ",len(maxVal))
    
    inp = []
    out = []
    numbRows = 1000
    for scene in batch:
#         if len(str(len(scene['lane']))) > 3:
#             print(len(scene['lane']))
#         print(scene['p_in'])
        lanes = numpy.zeros((numbRows,19,3))
        lane_norm = numpy.zeros((numbRows,19,3))
        pIn = vIn = numpy.zeros((numbRows,19,3))
#         lane_norm =[0,0]
        lengthLane = min(numbRows,len(scene['lane']))
        pIn[:len(scene['p_in']),:,:2] = scene['p_in']
        vIn[:len(scene['v_in']),:,:2] = scene['v_in']
        lanes[:lengthLane,0,:3] = scene['lane'][:lengthLane,:]
        lane_norm[:lengthLane,0,:3] = scene['lane_norm'] [:lengthLane,:]
        inp.append(numpy.dstack([pIn,vIn,lanes,lane_norm]))
        out.append(numpy.dstack([scene['p_out'], scene['v_out']]))
        agent_id= np.array([scene['agent_id'] for scene in batch],dtype=object)
        track_id= np.array([scene['track_id'][:,0] for scene in batch])
        
    inp = torch.sparse(torch.FloatTensor(inp))
    out = torch.sparse(torch.FloatTensor(out))
        
    return [inp, out,numpy.asarray(agent_id),numpy.asarray(track_id)]

train_loader = DataLoader(train_dataset,batch_size=batch_sz, shuffle = False, collate_fn=my_collate, num_workers=0)

In [ ]:

def val_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'], scene['v_in']]) for scene in batch]
    inp = torch.LongTensor(inp)
    return inp

val_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = False, collate_fn=my_collate, num_workers=0)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

class RNNModel(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers,dropoutVar=0.2):
        super(RNNModel, self).__init__()
        

        self.hidden_dim = hidden_dim
        self.num_layers = n_layers
        self.dropout = nn.Dropout(dropoutVar)
        self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True,dropout=0.5)
        self.fc = nn.Linear(hidden_dim, output_size)
    def forward(self, x,previous):
        with torch.cuda.amp.autocast(enabled=False):
            device = torch.device("cuda:0")
            x = x.to(device)
            h0 = 0
            c0 = 0
            if(previous == 1):
                h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim,device=device).requires_grad_()
                c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim,device=device).requires_grad_()
            else:
                hn,cn = previous
                h0 = hn
                c0 = cn
            out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
            out = self.fc(out[:, -1, :]) 
            
#             out = self.dropout(out)  ADD IF OVERFITTING
            return out,(hn,cn)
    
    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim,device=torch.device("cuda:0"))
        return hidden

In [ ]:
def make_a_histogram(sample_batch, agent_id, xPos, yPos, xVel, yVel):
    inp, out = sample_batch
    batch_sz = inp.size(0)
    #agent_sz = inp.size(1)
    
    for i in range(batch_sz):
        #hist_data_xPos = np.zeros((60,19));
        #hist_data_yPos = np.zeros((60,19));
        #hist_data_xVel = np.zeros((60,19));
        hist_data_yVel = np.zeros((60,19));
        
        for j in range(60):
            #hist_data_xPos[j] = (inp[i, j,:,0])
            #hist_data_yPos[j] = (inp[i, j,:,1])
            #hist_data_xVel[j] = (inp[i, j,:,2])
            hist_data_yVel[j] = (inp[i, j,:,3])
            
        for j in range(len(hist_data_yVel)):
            for k in range(len(hist_data_yVel[j])):
                #xPos.append(hist_data_xPos[j][k])
                #yPos.append(hist_data_yPos[j][k])
                #xVel.append(hist_data_xVel[j][k])
                yVel.append(hist_data_yVel[j][k])
    


In [ ]:
def show_sample_batch(sample_batch, agent_id):
    """visualize the trajectory for a batch of samples with a randon agent"""
    inp, out = sample_batch
    batch_sz = inp.size(0)
    agent_sz = inp.size(1)
    
    fig, axs = plt.subplots(1,batch_sz, figsize=(15, 3), facecolor='w', edgecolor='k')
    fig.subplots_adjust(hspace = .5, wspace=.001)
    axs = axs.ravel()   
    for i in range(batch_sz):
        axs[i].xaxis.set_ticks([])
        axs[i].yaxis.set_ticks([])
        
        # first two feature dimensions are (x,y) positions
        axs[i].scatter(inp[i, agent_id,:,0], inp[i, agent_id,:,1])
        axs[i].scatter(out[i, agent_id,:,0], out[i, agent_id,:,1])

### Visualize the batch of sequences

In [ ]:
import random
import numpy as np
import math

torch.cuda.empty_cache()
agent_id = 0
learning_rate = 1
momentum = 0.1
device = torch.device("cuda:0")
input_dim = 12    # input dimension
hidden_dim = 12  # hidden layer dimension
layer_dim = 10     # number of hidden layers
output_dim = 4   # output dimension

n_epochs = 5
lr=0.01

# Define Loss, Optimizer
#model = RNNModel(input_dim, output_dim, hidden_dim, layer_dim).to(device)
model = RNNModel(input_size=input_dim, output_size=output_dim, hidden_dim=hidden_dim, n_layers=layer_dim)
model = model.to(device)
# model = DataParallel(model,device)
optimizer = optim.Adagrad(model.parameters(), lr=learning_rate,lr_decay=0.00000001)
scaler = torch.cuda.amp.GradScaler()

    
model.train()
import time
print("test")
newOut = torch.zeros((batch_sz,1000,30,4))
batch = []
timeTotalLoss = 0
timeTotalModel = 0
timeEnumeration = 0
initHidden = 1
def my_loss(output, target,batch_num):
    loss = torch.mean((output[batch_num,:60] - target)**2)
    return loss

for i_epoch in range(n_epochs):
    timeStart = time.time()
    for i_batch, sample_batch in enumerate(train_loader):
        timeEnumeration = time.time() - timeStart
        timeStart = time.time()
    #     print("test")
        inp, out,agent_id,track_id = sample_batch
#         print(inp.shape)
        optimizer.zero_grad()
        scaled_loss = 0
        newOut[:,:60,:,:] = out
        for j in range(batch_sz):
#             print("agent_id",agent_id)
#             print("track_id",track_id)
            agentIndex = numpy.where(track_id[j]==agent_id[j])
            for i in range(30):

                with torch.cuda.amp.autocast():
                    timeStart = time.time()
                    output,hidden = model(inp[j].float().cuda(),1)
                    timeTotalModel += time.time() - timeStart
                    
                initHidden = hidden 
                hn,cn = initHidden
                
                
                x = inp[j,:60,:,:4]
                x = torch.roll(x,-1,dims=1)
                inp[j,:60,:19,:4] = x
                inp[j,:60,18,:4] = out[j,:60,i,:]
                
                x = output[agentIndex]
                output[:,:] = newOut[j,:,i,:]
#                 print(output)
                output[agentIndex]= x
                
                timeStart = time.time()
                loss = nn.MSELoss()
                loss = loss(output.cuda(),newOut[j,:,i,:].cuda())
#                 print(loss)
                scaler.scale(loss).backward(retain_graph=True)
                scaled_loss += loss.item()
                timeTotalLoss = time.time() - timeStart
        scaler.step(optimizer)
        batch.append(scaled_loss/(30 * batch_sz))
        scaler.update()
#         print("timeEnumeration",timeEnumeration)
#         print("timeTotalLoss ",timeTotalLoss)
#         print("timeTotalModel ",timeTotalModel)
        timeTotalLoss = timeTotalModel = 0
        if i_batch % math.floor(10) == 0:
            print("rowsSeen #: ",i_batch * batch_sz," avg loss:",mean(batch))
            batch = []

# from statistics import mean
# import random
# import numpy as np
# import math

# torch.cuda.empty_cache()
# agent_id = 0
# learning_rate = 1
# momentum = 0.1
# device = torch.device("cuda:0")
# input_dim = 12    # input dimension
# hidden_dim = 1  # hidden layer dimension
# layer_dim = 10     # number of hidden layers
# output_dim = 4   # output dimension

# n_epochs = 5
# lr=0.01

# # Define Loss, Optimizer
# #model = RNNModel(input_dim, output_dim, hidden_dim, layer_dim).to(device)
# model = RNNModel(input_size=input_dim, output_size=output_dim, hidden_dim=12, n_layers=1)
# model = model.to(device)
# # model = DataParallel(model,device)
# optimizer = optim.Adagrad(model.parameters(), lr=learning_rate,lr_decay=0.00000001)
# scaler = torch.cuda.amp.GradScaler()

    
# model.train()

# print("test")
# newOut = torch.zeros((batch_sz,100,30,4))
# batch = []
# for i_epoch in range(n_epochs):
#     for i_batch, sample_batch in enumerate(train_loader):
#     #     print("test")
#         inp, out = sample_batch
# #         print(inp[0])
#         optimizer.zero_grad()
#         initHidden = 1
#         scaled_loss = 0
#         newOut[:,:60,:,:] = out
#         for j in range(batch_sz):
#             for i in range(30):

#                 with torch.cuda.amp.autocast():
#                     output,hidden = model(inp[j].float().cuda(),initHidden)

#                 initHidden = hidden
#                 hn,cn = initHidden

# #                 import random
# #                 x = random.uniform(0, 1)
# #                 if x < 0.5:
# #                     continue
#                 loss = nn.MSELoss()
#                 loss = loss(output.cuda(),newOut[j,:,i,:].cuda())
#                 scaler.scale(loss.to(torch.float16)).backward(retain_graph=True)
#                 scaled_loss += loss.item()
#         scaler.step(optimizer)
#         batch.append(scaled_loss/(30 * batch_sz))
#         scaler.update()
#         if i_batch % math.floor(100) == 0:
#             print("rowsSeen #: ",i_batch * batch_sz," avg loss (past 100): ",mean(batch))
#             batch = []



test
rowsSeen #:  0  avg loss: 1272.8967134277025
rowsSeen #:  40  avg loss: 1035.4652445566653
rowsSeen #:  80  avg loss: 1242.164486468633
rowsSeen #:  120  avg loss: 1004.4546915501853
rowsSeen #:  160  avg loss: 1457.476577326705
rowsSeen #:  200  avg loss: 1653.6072672683001
rowsSeen #:  240  avg loss: 928.4885604109119
rowsSeen #:  280  avg loss: 899.8442958218604
rowsSeen #:  320  avg loss: 868.9512193649914
rowsSeen #:  360  avg loss: 1012.308119630913
rowsSeen #:  400  avg loss: 665.5177795279462
rowsSeen #:  440  avg loss: 701.2004108149434
rowsSeen #:  480  avg loss: 711.8553521788082
rowsSeen #:  520  avg loss: 1184.820278878975
rowsSeen #:  560  avg loss: 973.4908709893344
rowsSeen #:  600  avg loss: 1047.0313925888963
rowsSeen #:  640  avg loss: 959.7693649898845
rowsSeen #:  680  avg loss: 924.7784717907718
rowsSeen #:  720  avg loss: 1126.9705387735796
rowsSeen #:  760  avg loss: 906.8472390528925
rowsSeen #:  800  avg loss: 979.3896255665583
rowsSeen #:  840  avg loss:

rowsSeen #:  7000  avg loss: 731.3182914061969
rowsSeen #:  7040  avg loss: 700.1863346724201
rowsSeen #:  7080  avg loss: 581.8241695577154
rowsSeen #:  7120  avg loss: 480.5255182314757
rowsSeen #:  7160  avg loss: 930.9731729666391
rowsSeen #:  7200  avg loss: 804.6898390885318
rowsSeen #:  7240  avg loss: 594.9150288103397
rowsSeen #:  7280  avg loss: 761.4298697069288
rowsSeen #:  7320  avg loss: 547.5769667260969
rowsSeen #:  7360  avg loss: 698.5507729133033
rowsSeen #:  7400  avg loss: 1075.600728502497
rowsSeen #:  7440  avg loss: 532.2791401253703
rowsSeen #:  7480  avg loss: 946.2446402196834
rowsSeen #:  7520  avg loss: 486.3313694216994
rowsSeen #:  7560  avg loss: 918.9290096339374
rowsSeen #:  7600  avg loss: 727.7429943418648
rowsSeen #:  7640  avg loss: 674.8909095731648
rowsSeen #:  7680  avg loss: 683.3149171889067
rowsSeen #:  7720  avg loss: 754.1170167874669
rowsSeen #:  7760  avg loss: 631.4553576591859
rowsSeen #:  7800  avg loss: 726.8831010436378
rowsSeen #:  

In [ ]:
import torch 
print(torch.__version__)